# Explanation of Decoding:
**orderHash**: Unique identifier for each order.

**maker**: Address of the maker (the person who placed the order).

**taker**: Address of the taker (the person who filled the order).

**makerAssetId**: The asset ID that the maker is offering.

**takerAssetId**: The asset ID that the taker is offering.

**makerAmountFilled**: The amount of the maker's asset that was filled.

**takerAmountFilled**: The amount of the taker's asset that was filled.

**fee**: Any fees associated with the transaction.

**event**: The type of event (OrderFilled, OrdersMatched, etc.).

**takerOrderHash**: The order hash of the taker's order (if applicable).

**takerOrderMaker**: The address of the taker's order maker (if applicable).

### Token Registered Specific
**token0**, **token1**, **conditionId**: Related to TokenRegistered events, representing tokens involved and the condition ID.

In [5]:
import csv
from web3 import Web3
from eth_abi import decode
import pandas as pd

from web3 import Web3

event_signatures = {
    'OrderFilled': 'OrderFilled(bytes32,address,address,uint256,uint256,uint256,uint256,uint256)',
    'OrdersMatched': 'OrdersMatched(bytes32,address,uint256,uint256,uint256,uint256)',
    'PositionsMerge': 'PositionsMerge(address,address,bytes32,bytes32,uint256[],uint256)',
    'TokenRegistered': 'TokenRegistered(uint256,uint256,bytes32)',

}

event_topic0 = {name: Web3.keccak(text=signature).hex() for name, signature in event_signatures.items()}

print(event_topic0)


def decode_address(topic):
    return '0x' + topic[-40:]

def decode_uint256(data):
    return int(data, 16)

def decode_order_filled(log):
    decoded_event = {}
    decoded_event['orderHash'] = log['topic1']
    decoded_event['maker'] = decode_address(log['topic2'])
    decoded_event['taker'] = decode_address(log['topic3'])
    data = log['data']
    chunks = [data[i:i+64] for i in range(0, len(data), 64)]
    decoded_event['makerAssetId'] = decode_uint256(chunks[0])
    decoded_event['takerAssetId'] = decode_uint256(chunks[1])
    decoded_event['makerAmountFilled'] = decode_uint256(chunks[2])
    decoded_event['takerAmountFilled'] = decode_uint256(chunks[3])
    decoded_event['fee'] = decode_uint256(chunks[4]) if len(chunks) > 4 else 0
    return decoded_event

def decode_orders_matched(log):
    decoded_event = {}
    decoded_event['takerOrderHash'] = log['topic1']
    decoded_event['takerOrderMaker'] = decode_address(log['topic2'])
    data = log['data']
    chunks = [data[i:i+64] for i in range(0, len(data), 64)]
    decoded_event['makerAssetId'] = decode_uint256(chunks[0])
    decoded_event['takerAssetId'] = decode_uint256(chunks[1])
    decoded_event['makerAmountFilled'] = decode_uint256(chunks[2])
    decoded_event['takerAmountFilled'] = decode_uint256(chunks[3])
    return decoded_event

def decode_positions_merged(log):
    decoded_event = {}
    # Indexed parameters
    decoded_event['stakeholder'] = decode_address(log['topic1'])
    decoded_event['parentCollectionId'] = log['topic2']
    decoded_event['conditionId'] = log['topic3']
    # Non-indexed parameters
    data = log['data']
    data_bytes = bytes.fromhex(data[2:])  # Skip '0x' prefix
    # Decode the parameters using eth_abi
    types = ['address', 'uint256[]', 'uint256']
    decoded_values = decode(types, data_bytes)
    decoded_event['collateralToken'] = decoded_values[0]
    decoded_event['partition'] = decoded_values[1]
    decoded_event['amount'] = decoded_values[2]
    return decoded_event

def decode_token_registered(log):
    decoded_event = {}
    # Indexed parameters
    decoded_event['token0'] = int(log['topic1'], 16)
    decoded_event['token1'] = int(log['topic2'], 16)
    decoded_event['conditionId'] = log['topic3']
    # There is no data field to decode
    return decoded_event


# Read log entries
log_entries = []
total_log_entries = 0

for i in range(33, 61):
        
    with open(f'ctf_exchange/logs_{i}M.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            log_entries.append(row)

    # Decode logs
    decoded_events = []
    non_decoded_events = []

    for log in log_entries:
        topic0 = log['topic0']
        total_log_entries += 1
        if topic0 == '0x' + event_topic0['OrderFilled']:
            decoded_event = decode_order_filled(log)
            decoded_event['event'] = 'OrderFilled'
        elif topic0 == '0x' + event_topic0['OrdersMatched']:
            decoded_event = decode_orders_matched(log)
            decoded_event['event'] = 'OrdersMatched'
        elif topic0 == '0x' + event_topic0['PositionsMerge']:
            decoded_event = decode_positions_merged(log)
            decoded_event['event'] = 'PositionsMerge'
        elif topic0 == '0x' + event_topic0['TokenRegistered']:
            decoded_event = decode_token_registered(log)
            decoded_event['event'] = 'TokenRegistered'

        else:
            if (non_decoded_events.__contains__(topic0)):
                continue
            non_decoded_events.append(topic0)
            continue
        decoded_events.append(decoded_event)
        



{'OrderFilled': 'd0a08e8c493f9c94f29311604c9de1b4e8c8d4c06bd0c789af57f2d65bfec0f6', 'OrdersMatched': '63bf4d16b7fa898ef4c4b2b6d90fd201e9c56313b65638af6088d149d2ce956c', 'PositionsMerge': '6f13ca62553fcc2bcd2372180a43949c1e4cebba603901ede2f4e14f36b282ca', 'TokenRegistered': 'bc9a2432e8aeb48327246cddd6e872ef452812b4243c04e6bfb786a2cd8faf0d'}


In [8]:
print('Total Entries: ' + f'{total_log_entries}')
print('To Decode Events: ' + f'{len(non_decoded_events)}')
print(non_decoded_events)

df = pd.DataFrame(decoded_events)
df.to_csv('decoded_events.csv', index=False)

Total Entries: 18895210
To Decode Events: 12
['0xf9ffabca9c8276e99321725bcb43fb076a6c66a54b7f21c4e8146d8519b417dc', '0xf1e04d73c4304b5ff164f9d10c7473e2a1593b740674a6107975e2a7001c1e5c', '0x787a2e12f4a55b658b8f573c32432ee11a5e8b51677d1e1e937aaf6a0bb5776e', '0xf7262ed0443cc211121ceb1a80d69004f319245615a7488f951f1437fd91642c', '0xacffcc86834d0f1a64b0d5a675798deed6ff0bcfc2231edd3480e7288dba7ff4', '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef', '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0xc3d58168c5ae7397731d063d5bbf3d657854427343f4c083240f7aacaa2d0f62', '0x4a39dc06d4c0dbc64b70af90fd698a233a518aa5d07e595d983b8c0526c8f7fb', '0x2e6bb91f8cbcda0c93623c54d0403a43514fabc40084ec96b6d5379a74786298', '0x4dfe1bbbcf077ddc3e01291eea2d5c70c2b422b415d95645b9adcfd678cb1d63', '0x18fe0464eb77016dc4e227eb0d690e4002756d82b442143bbfb874548952b5f2']
